In [18]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as img
import numpy as np
import cv2
import os, shutil
from scipy.spatial import distance
import xml.etree.ElementTree as ET

### Annotation 파일 읽어오기
https://codechacha.com/ko/python-parsing-xml-and-pretty-print/

In [19]:
import os 

path = './extend'
annotation_list = os.listdir(path) 
annotation_list.remove('images')
print(annotation_list)

['annotations.xml']


### 함수 클래스 정의

In [20]:
class Utils:
    def __init__(self):
        pass
    
    # 각 행의 n 번째 열을 기준으로 sort (오름차순)
    # x : nd array
    # axis : 기준 열
    def sort_ndarray_by_col(self, x, axis=0):
        size = x.shape[0]
        col = [x[i][axis] for i in range(size)] # 기준 행의 원소들을 추출
        sorted_col = sorted(col) # 추출된 원소들을 정렬
        rank = [sorted_col.index(x[i][axis]) for i in range(size)] # 정렬된 index 추출
        #print(rank)
        try:
            sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬
        except: # 똑같은 순위가 있는 경우 예외처리
            overlap_rank = []
            for i in range(size):
                if rank.count(i) > 1:
                    overlap_rank.append(i)
            for r in overlap_rank:
                overlap_list = [i for i in range(size) if rank[i]==r]
                for k, idx in enumerate(overlap_list):
                    rank[idx] += k
            sorted_array = [x[rank.index(i)] for i in range(size)]  # 정렬된 index 순서대로 정렬

        return np.array(sorted_array)

    # 함수화
    def perspective_transform(self, src, point, height=256, width=256):    
        # 4 꼭지점 분류 
        sorted_by_y= self.sort_ndarray_by_col(point, 1)
        top_two = sorted_by_y[0:2]
        topLeft, topRight = self.sort_ndarray_by_col(top_two, 0)
        bottom_two = sorted_by_y[2:4]
        bottomLeft, bottomRight = self.sort_ndarray_by_col(bottom_two, 0)
        #print("tl:", topLeft)
        #print("bl", bottomLeft)
        #print("tr", topRight)
        #print("br", bottomRight)


        # 변환 전 4개 좌표 
        srcPoint = np.float32([topLeft, topRight, bottomRight , bottomLeft])

        # 변환 후 4개 좌표
        dstPoint = np.array([[0, 0], [width, 0], [width, height], [0, height]], dtype=np.float32)

        # Perspective transformation
        matrix = cv2.getPerspectiveTransform(srcPoint, dstPoint)
        dst = cv2.warpPerspective(src, matrix, (width, height))

        return dst

    # 이미지 상에 polygon 그리기
    def draw_label(self, img_path, points, save_path=""):
        '''
        image = img.imread(img_path)
        plt.imshow(image)

        ax = plt.gca()

        point = np.array([[data['p1'][0], data['p1'][1]],
                          [data['p2'][0], data['p2'][1]],
                          [data['p3'][0], data['p3'][1]],
                          [data['p4'][0], data['p4'][1]]])
        #print(point)

        polygon = patches.Polygon(point, fill=None ,edgecolor='k',ls='solid',lw=3)

        ax.add_patch(polygon)

        plt.show()
        '''
        src = cv2.imread(img_path, cv2.IMREAD_COLOR)
        dst = cv2.polylines(src, np.int32([points]), True, (255, 0, 0), 6)

        return dst

    def extract_bb_points(self, points, width, height):
        if len(points.split(';')) == 4: # 라벨링된 point가 4개인 경우 = polygon 라벨링 (없음)
            ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                 (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1])),
                 (float(points.split(';')[2].split(',')[0]), float(points.split(';')[2].split(',')[1])),
                 (float(points.split(';')[3].split(',')[0]), float(points.split(';')[3].split(',')[1]))]

            # 4 꼭지점 분류 
            sorted_by_y= self.sort_ndarray_by_col(np.array(ps), 1)
            top_two = sorted_by_y[0:2]
            topLeft, topRight = self.sort_ndarray_by_col(top_two, 0)
            bottom_two = sorted_by_y[2:4]
            bottomLeft, bottomRight = list(self.sort_ndarray_by_col(bottom_two, 0))

        elif len(points.split(';')) == 2: # 라벨링된 point가 2개인 경우 = 직선 라벨링 (있음))
            ps = [(float(points.split(';')[0].split(',')[0]), float(points.split(';')[0].split(',')[1])),
                 (float(points.split(';')[1].split(',')[0]), float(points.split(';')[1].split(',')[1]))]

            # 2 꼭지점 분류
            if ps[0][0] < ps[1][0]:
                bottomLeft = ps[0]
                bottomRight = ps[1]
            else:
                bottomLeft = ps[1]
                bottomRight = ps[0]
        else:
            return None, 0, None

        print(len(points.split(';')))
        print('bb', bottomLeft, bottomRight) # bounding box 생성에 기준이 되는 주차면 앞 라인의 두 점

        # 라인으로부터 바운딩 박스 생성
        x_offset = 0
        width = distance.euclidean(bottomLeft, bottomRight)+x_offset
        y_offset = width/10.0
        height = -width*2/3.0
        #print(ps, width, height)


        b1 = (bottomLeft[0], bottomLeft[1]+y_offset)
        b2 = (bottomRight[0], bottomRight[1]+y_offset)
        b3 = (b2[0], b2[1]+height)
        b4 = (b1[0], b1[1]+height)
        print('ps:',np.array([b1, b2, b3, b4]), width, np.array(ps))

        return np.array([b1, b2, b3, b4]), width, np.array(ps)

    def get_occupied_info(self, polyline, width, height):
        label = polyline.get('label')
        points = polyline.get('points') # 2개
        bb_points, width, _ = self.extract_bb_points(points, width, height)
        return label, width, bb_points

    def get_empty_info(self, polygon, width, height):
        label = polygon.get('label')
        points = polygon.get('points') # 4개 or 5개
        bb_points, width, poly_points = self.extract_bb_points(points, width, height)
        empty_type_dict = {'일반형':0, '경형':1, '장애인전용':2, '여성우선주차장':3, '환경친화적 자동차 전용':4, 'etc':5, '기타':5}
        empty_type = empty_type_dict[polygon.find('attribute').text]
        return label, width, bb_points, poly_points, empty_type

    def get_obstacle_info(self, polygon, width, height):
        label = polygon.get('label')
        points = polygon.get('points')
        bb_points, width, poly_points = self.extract_bb_points(points, width, height)
        obstacle_type_dict = {'사람':0, '이륜차':1, '기타':2}
        obs_type = obstacle_type_dict[polygon.find('attribute').text]
        return label, width, bb_points, poly_points, obs_type

In [21]:
#from utils import Utils
# 함수 클래스 인스턴스화
ut = Utils()

### 데이터셋 리스트 생성

In [22]:
dataset_list = []
for annotation in annotation_list:
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    tree = ET.parse(filePath)
    root = tree.getroot()
    dataset_name = root.find("meta").find("task").find("name").text
    dataset_list.append(dataset_name)
    print(annotation, dataset_name)

annotations.xml Extended_Car


### xml 파일로부터 라벨링 정보 추출

In [23]:
occupied_cnt, empty_cnt, obstacle_cnt = 0, 0, 0

datasets = {}
datasets['occupied'] = {}
datasets['empty'] = {}
datasets['obstacle'] = {}

for annotation in annotation_list:
    print(annotation)
    # annotation 파일 불러오기
    filePath = os.path.join(path, annotation)
    tree = ET.parse(filePath)
    root = tree.getroot()
    dataset_name = root.find("meta").find("task").find("name").text
    
    for image in root.findall("image"):
        image_attrib = image.attrib
        image_frame = image_attrib.get('id')
        image_name = image_attrib.get('name')
        print(image_name)
        if not os.path.exists(os.path.join(path, 'images', image_name)): # 이미지가 존재하지 않을 경우 pass
            continue
        image_witdh = image_attrib.get('width')
        image_height = image_attrib.get('height')
        polylines = image.findall('polyline') # 있음 라벨링
        polygons = image.findall('polygon') # 없음 라벨링
        
        for polyline in polylines: # 있음
            polyline_label, width, bb_points = ut.get_occupied_info(polyline, image_witdh, image_height)
            print(polyline_label) 
            if width > 250: # 길이가 너무 짧은 경우 넘어감
                occupied_cnt += 1 
                datasets['occupied'][str(occupied_cnt)]={'label':polyline_label, 'src':image_name,
                                                         'dataset':dataset_name,
                                                         'frame':image_frame,
                                                         'width': width,
                                                         'bb_points':bb_points
                                                        }
            
        for polygon in polygons:
            label = polygon.get('label')
            if label == '없음': # 없음
                empty_label, width, bb_points, poly_points, empty_type = ut.get_empty_info(polygon, image_witdh, image_height)
                print(empty_label, empty_type)
                if (type(bb_points) != None) and (width > 250): # point 정보가 없거나 길이가 너무 짧은 경우 넘어감
                    empty_cnt += 1
                    datasets['empty'][str(empty_cnt)]={'label':empty_label, 'src':image_name,
                                                       'dataset':dataset_name,
                                                       'frame':image_frame,
                                                       'width':width,
                                                       'type':empty_type,
                                                       'bb_points':bb_points,
                                                       'poly_points':poly_points
                                                      }
                
            elif label == '장애물': # 장애물
                obs_label, width, bb_points, poly_points, obs_type = ut.get_obstacle_info(polygon, image_witdh, image_height)
                print(obs_label, obs_type)
                obstacle_cnt += 1
                datasets['obstacle'][str(obstacle_cnt)]={'label':obs_label, 'src':image_name,
                                                         'dataset':dataset_name,
                                                         'frame':image_frame,
                                                         'width': width,
                                                         'type':obs_type,
                                                         'poly_points':poly_points
                                                        }
                
        #occupied_cnt += len(polylines)
        #empty_cnt += len(polygons)
        #print(image_name, '| 있음', len(polylines), '없음', len(polygons))
    

annotations.xml
extend1.jpg
4
bb [ 735.8  1752.88] [1126.12 1650.45]
ps: [[ 735.8        1793.23363767]
 [1126.12       1690.80363767]
 [1126.12       1421.77938655]
 [ 735.8        1524.20938655]] 403.53637667501545 [[ 850.6  1341.37]
 [ 935.39 1327.28]
 [1126.12 1650.45]
 [ 735.8  1752.88]]
장애물 2
extend10.jpg
4
bb [ 737.56 1756.41] [1127.88 1653.98]
ps: [[ 737.56       1796.76363767]
 [1127.88       1694.33363767]
 [1127.88       1425.30938655]
 [ 737.56       1527.73938655]] 403.5363766750157 [[ 852.36 1344.9 ]
 [ 937.15 1330.81]
 [1127.88 1653.98]
 [ 737.56 1756.41]]
장애물 2
extend100.jpg
4
bb [2894.98 1510.14] [3076.89 1529.57]
ps: [[2894.98       1528.43447266]
 [3076.89       1547.86447266]
 [3076.89       1425.90132158]
 [2894.98       1406.47132158]] 182.94472662528412 [[2963.86 1245.22]
 [3034.5  1257.58]
 [3076.89 1529.57]
 [2894.98 1510.14]]
장애물 2
extend1000.jpg
extend1001.jpg
extend1002.jpg
extend1003.jpg
extend1004.jpg
extend1005.jpg
extend1006.jpg
extend1007.jpg
extend1008

4
bb [1482.06 1480.11] [1692.23 1485.41]
ps: [[1482.06       1501.13368162]
 [1692.23       1506.43368162]
 [1692.23       1366.27580413]
 [1482.06       1360.97580413]] 210.23681623350376 [[1558.   1079.2 ]
 [1688.7  1079.2 ]
 [1692.23 1485.41]
 [1482.06 1480.11]]
장애물 0
4
bb [1268.35 1573.72] [1446.73 1552.53]
ps: [[1268.35       1591.68341852]
 [1446.73       1570.49341852]
 [1446.73       1450.73729504]
 [1268.35       1471.92729504]] 179.63418522096524 [[1275.42 1245.22]
 [1422.01 1238.15]
 [1446.73 1552.53]
 [1268.35 1573.72]]
장애물 2
4
bb [1167.68 1571.95] [1268.35 1575.49]
ps: [[1167.68       1582.02322218]
 [1268.35       1585.56322218]
 [1268.35       1518.40840767]
 [1167.68       1514.86840767]] 100.73222175649641 [[1144.72 1331.76]
 [1245.39 1321.16]
 [1268.35 1575.49]
 [1167.68 1571.95]]
장애물 2
4
bb [2117.87 1566.66] [2391.62 1568.42]
ps: [[2117.87       1594.03556577]
 [2391.62       1595.79556577]
 [2391.62       1413.29179399]
 [2117.87       1411.53179399]] 273.7556576584

4
bb [1589.07 2160.  ] [2527.31 2160.  ]
ps: [[1589.07       2253.824     ]
 [2527.31       2253.824     ]
 [2527.31       1628.33066667]
 [1589.07       1628.33066667]] 938.24 [[1500.1  1434.18]
 [2369.89 1296.84]
 [2527.31 2160.  ]
 [1589.07 2160.  ]]
장애물 2
4
bb [2614.04 2059.82] [3541.8 1977.4]
ps: [[2614.04       2152.96138038]
 [3541.8        2070.54138038]
 [3541.8        1449.59884449]
 [2614.04       1532.01884449]] 931.4138038487514 [[2354.63 1251.06]
 [2885.66 1229.7 ]
 [3541.8  1977.4 ]
 [2614.04 2059.82]]
장애물 2
extend339.jpg
4
bb [1589.07 2160.  ] [2527.31 2160.  ]
ps: [[1589.07       2253.824     ]
 [2527.31       2253.824     ]
 [2527.31       1628.33066667]
 [1589.07       1628.33066667]] 938.24 [[1500.1  1434.18]
 [2369.89 1296.84]
 [2527.31 2160.  ]
 [1589.07 2160.  ]]
장애물 2
4
bb [2614.04 2059.82] [3541.8 1977.4]
ps: [[2614.04       2152.96138038]
 [3541.8        2070.54138038]
 [3541.8        1449.59884449]
 [2614.04       1532.01884449]] 931.4138038487514 [[2354.63 1

4
bb [ 229.86 1268.18] [ 651.97 1298.2 ]
ps: [[ 229.86       1310.49761483]
 [ 651.97       1340.51761483]
 [ 651.97       1058.40018262]
 [ 229.86       1028.38018262]] 423.17614831178753 [[ 228.09 1231.09]
 [ 229.86 1268.18]
 [ 651.97 1298.2 ]
 [ 655.5  1252.28]]
장애물 2
4
bb [ 579.56 1504.84] [ 798.56 1465.99]
ps: [[ 579.56       1527.08192494]
 [ 798.56       1488.23192494]
 [ 798.56       1339.95242534]
 [ 579.56       1378.80242534]] 222.41924939177363 [[ 636.07 1248.75]
 [ 736.74 1229.32]
 [ 798.56 1465.99]
 [ 579.56 1504.84]]
장애물 2
4
bb [ 746.28 1292.02] [1163.09 1288.49]
ps: [[ 746.28       1333.70249477]
 [1163.09       1330.17249477]
 [1163.09       1052.28919632]
 [ 746.28       1055.81919632]] 416.8249476698821 [[1162.38 1255.81]
 [1163.09 1288.49]
 [ 746.28 1292.02]
 [ 730.38 1237.27]]
장애물 2
4
bb [1100.57 1518.97] [1316.04 1504.84]
ps: [[1100.57       1540.56328085]
 [1316.04       1526.43328085]
 [1316.04       1382.47807517]
 [1100.57       1396.60807517]] 215.93280853080

4
bb [1074.08 1358.25] [1422.01 1218.72]
ps: [[1074.08       1395.73651835]
 [1422.01       1256.20651835]
 [1422.01       1006.29639602]
 [1074.08       1145.82639602]] 374.86518349934823 [[1116.46  854.9 ]
 [1441.44  948.5 ]
 [1422.01 1218.72]
 [1074.08 1358.25]]
장애물 1
4
bb [1743.45 1222.26] [1902.4  1224.02]
ps: [[1743.45       1238.15597436]
 [1902.4        1239.91597436]
 [1902.4        1133.94281193]
 [1743.45       1132.18281193]] 158.95974364599363 [[1927.13  895.52]
 [1716.96  918.48]
 [1743.45 1222.26]
 [1902.4  1224.02]]
장애물 1
4
bb [2054.29 1218.72] [2199.11 1201.06]
ps: [[2054.29       1233.30927963]
 [2199.11       1215.64927963]
 [2199.11       1118.38741545]
 [2054.29       1136.04741545]] 145.89279625807455 [[2188.52  856.66]
 [1971.28  890.22]
 [2054.29 1218.72]
 [2199.11 1201.06]]
장애물 1
4
bb [2243.27 1091.56] [2539.98 1254.05]
ps: [[2243.27       1125.38895567]
 [2539.98       1287.87895567]
 [2539.98       1062.35258451]
 [2243.27        899.86258451]] 338.2895567409

In [24]:
datasets

{'occupied': {},
 'empty': {},
 'obstacle': {'1': {'label': '장애물',
   'src': 'extend1.jpg',
   'dataset': 'Extended_Car',
   'frame': '0',
   'width': 403.53637667501545,
   'type': 2,
   'poly_points': array([[ 850.6 , 1341.37],
          [ 935.39, 1327.28],
          [1126.12, 1650.45],
          [ 735.8 , 1752.88]])},
  '2': {'label': '장애물',
   'src': 'extend10.jpg',
   'dataset': 'Extended_Car',
   'frame': '1',
   'width': 403.5363766750157,
   'type': 2,
   'poly_points': array([[ 852.36, 1344.9 ],
          [ 937.15, 1330.81],
          [1127.88, 1653.98],
          [ 737.56, 1756.41]])},
  '3': {'label': '장애물',
   'src': 'extend100.jpg',
   'dataset': 'Extended_Car',
   'frame': '2',
   'width': 182.94472662528412,
   'type': 2,
   'poly_points': array([[2963.86, 1245.22],
          [3034.5 , 1257.58],
          [3076.89, 1529.57],
          [2894.98, 1510.14]])},
  '4': {'label': '장애물',
   'src': 'extend101.jpg',
   'dataset': 'Extended_Car',
   'frame': '13',
   'width': 526.

### Patch 정보 확인

In [25]:
datasets['obstacle']['1152']

{'label': '장애물',
 'src': 'extend322.jpg',
 'dataset': 'Extended_Car',
 'frame': '678',
 'width': 876.25412295749,
 'type': 2,
 'poly_points': array([[1672.8 , 1294.67],
        [2294.5 , 1257.6 ],
        [2257.4 , 2047.05],
        [1384.92, 1965.81]])}

### 카테고리별 개수 확인

In [26]:
len(datasets['occupied']), len(datasets['empty']), len(datasets['obstacle'])

(0, 0, 2415)

In [27]:
len(datasets['occupied']) + len(datasets['empty']) + len(datasets['obstacle'])

2415

### 세부구획 개수

In [28]:
num_empty = len(datasets['empty'])
print("총 라벨링된 세부구획 개수: {}".format(num_empty))

총 라벨링된 세부구획 개수: 0


In [29]:
num_empty_type_list = [0, 0, 0, 0, 0, 0] # 일반형, 경형, 장애인전용, 여성우선주차장, 환경친화적 자동차 전용, 기타
for key in datasets['empty'].keys(): # frame 1,2,3,...
    empty_type = datasets['empty'][key]['type'] # 세부구획 0 1 2 3 4 5
    num_empty_type_list[empty_type] += 1
print("세부구획 개수 : 일반형 {} 경형 {} 장애인전용 {} 여성우선주차장 {} 환경친화적 자동차 전용 {} 기타 {}".format(*num_empty_type_list))

세부구획 개수 : 일반형 0 경형 0 장애인전용 0 여성우선주차장 0 환경친화적 자동차 전용 0 기타 0


### 장애물 종류 개수

In [30]:
num_obstacle = len(datasets['obstacle'])
print("총 라벨링된 장애물 개수: {}".format(num_obstacle))

총 라벨링된 장애물 개수: 2415


In [31]:
num_obs_list = [0, 0, 0] # 일반형 이륜차 기타
for key in datasets['obstacle'].keys():
    obs_type = datasets['obstacle'][key]['type']
    num_obs_list[obs_type] += 1
print("장애물 종류 개수 : 사람 {} 이륜차 {} 기타 {}".format(*num_obs_list))

장애물 종류 개수 : 사람 38 이륜차 280 기타 2097


### 각 주차장 데이터셋 당 개수

In [32]:
# 개수 정보를 저장할 딕셔너리 초기화
num_dataset = {}
for dataset in dataset_list:
    num_dataset[dataset] = {'occupied':0, 'empty':0, 'obstacle': 0}

# 각 데이터셋마다 데이터 수 누적
for category in datasets.keys(): # occupied empty obstacle
    for data in datasets[category].keys(): # frame 1,2,3,4...
        dataset = datasets[category][data]['dataset'] # 데이터셋 이름 ex '어바니엘 가산'
        num_dataset[dataset][category] += 1 # 데이터 수 카운트

In [33]:
num_dataset

{'Extended_Car': {'occupied': 0, 'empty': 0, 'obstacle': 2415}}